<a href="https://colab.research.google.com/github/shubhamgundawarNYU/Big-Data-Project-Group-16/blob/main/misc-datasets-notebooks/NYPD_Hate_Crimes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**BIG DATA PROJECT**

### NYPD Hate Crimes
Link to Dataset (https://data.cityofnewyork.us/Public-Safety/NYPD-Hate-Crimes/bqiq-cu78)

### DATA CLEANING AT SCALE

#### Mounting Google Drive to Google Collab Notebook to Load the Data Set

Make sure you have the dataset in your Google Drive and you mount your drive to the Colab.

The file should be at the following path: `gdrive/My Drive/NYPD_Complaint_Data_Historic.csv`


In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


#### Importing required and Necessary Libraries for cleaning the data present in the data set

In [ ]:
import numpy as np
import pandas as pd
import io

In [ ]:
!pip install pyspark

     |████████████████████████████████| 281.3 MB 38 kB/s 
     |████████████████████████████████| 198 kB 53.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=1ee2480096e8b6a105944fc42195db2c6f42117dbfcc40d67cae3a5f5cf8bb75
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark



# **Running Pyspark in Colab**

To run spark in Colab, we need to first install all the dependencies in Colab environment i.e. Apache Spark 2.3.2 with hadoop 2.7, Java 8 and Findspark to locate the spark in the system. The tools installation can be carried out inside the Jupyter Notebook of the Colab. One important note is that if you are new in Spark, it is better to avoid Spark 2.4.0 version since some people have already complained about its compatibility issue with python. 
Follow the steps to install the dependencies:

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz
!tar xf spark-3.2.0-bin-hadoop3.2.tgz
!pip install -q findspark

Now that you installed Spark and Java in Colab, it is time to set the environment path which enables you to run Pyspark in your Colab environment. Set the location of Java and Spark by running the following code:

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop3.2"

Run a local spark session to test your installation:

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
spark = SparkSession.builder.getOrCreate()

#### Reading the Data Set CSV File using `spark.read.csv()` Function

In [ ]:
df = spark.read.csv("/content/gdrive/MyDrive/Big Data/NYPD_Hate_Crimes.csv", inferSchema=True, header =True)

In [ ]:
df.count()

1181

#### Get Data Type for each column present in the Data Set




In [ ]:
df.printSchema()

root
 |-- Full Complaint ID: long (nullable = true)
 |-- Complaint Year Number: integer (nullable = true)
 |-- Month Number: integer (nullable = true)
 |-- Record Create Date: string (nullable = true)
 |-- Complaint Precinct Code: integer (nullable = true)
 |-- Patrol Borough Name: string (nullable = true)
 |-- County: string (nullable = true)
 |-- Law Code Category Description: string (nullable = true)
 |-- Offense Description: string (nullable = true)
 |-- PD Code Description: string (nullable = true)
 |-- Bias Motive Description: string (nullable = true)
 |-- Offense Category: string (nullable = true)
 |-- Arrest Date: string (nullable = true)
 |-- Arrest Id: string (nullable = true)



#### Outputing the List of Columns in the Data Set

In [ ]:
df.columns

['Full Complaint ID',
 'Complaint Year Number',
 'Month Number',
 'Record Create Date',
 'Complaint Precinct Code',
 'Patrol Borough Name',
 'County',
 'Law Code Category Description',
 'Offense Description',
 'PD Code Description',
 'Bias Motive Description',
 'Offense Category',
 'Arrest Date',
 'Arrest Id']

#### Get top 10 rows of the complaints dataframe

In [ ]:
df.show(n=10)

+-----------------+---------------------+------------+------------------+-----------------------+-------------------+------+-----------------------------+--------------------+--------------------+-----------------------+--------------------+-----------+---------+
|Full Complaint ID|Complaint Year Number|Month Number|Record Create Date|Complaint Precinct Code|Patrol Borough Name|County|Law Code Category Description| Offense Description| PD Code Description|Bias Motive Description|    Offense Category|Arrest Date|Arrest Id|
+-----------------+---------------------+------------+------------------+-----------------------+-------------------+------+-----------------------------+--------------------+--------------------+-----------------------+--------------------+-----------+---------+
|  201904612204817|                 2019|           2|        02/08/2019|                     46|  PATROL BORO BRONX| BRONX|                       FELONY|      FELONY ASSAULT|ASSAULT 2,1,UNCLA...|   ANTI-MALE

In [ ]:
df.columns

['Full Complaint ID',
 'Complaint Year Number',
 'Month Number',
 'Record Create Date',
 'Complaint Precinct Code',
 'Patrol Borough Name',
 'County',
 'Law Code Category Description',
 'Offense Description',
 'PD Code Description',
 'Bias Motive Description',
 'Offense Category',
 'Arrest Date',
 'Arrest Id']

#### Removing all the **duplicate** entries

In [ ]:
df = df.drop_duplicates()

In [ ]:
df.count()

1181

In [ ]:
df.distinct().count()

1181

[link text](https://)#### **Checking** if the Full Complaint ID is unique or not

In [ ]:
df.select('Full Complaint ID').distinct().count()

1102

#### As we can see `CMPLNT_NUM` should have been unique, but it is not.
#### Let's see what are the duplicate values.

In [ ]:
df1 = df.groupBy('Full Complaint ID').count().filter("count > 1")
df1.drop('count').count()

58

In [ ]:
df1.sort('Full Complaint ID').show(n = 10)

+-----------------+-----+
|Full Complaint ID|count|
+-----------------+-----+
|  201900912215017|    2|
|  201901812178017|    2|
|  201902012074117|    2|
|  201902512693417|    3|
|  201904212405517|    3|
|  201904313084517|    2|
|  201906612592617|    4|
|  201907112129217|    3|
|  201907612245617|    3|
|  201907912598517|    2|
+-----------------+-----+
only showing top 10 rows



#### We understand, that complaint number is not specifically unique. The dataset has rows with duplicated complaint numbers having unique information for all other columns. Thus, we cannot drop the entries with duplicated complaint numbers.

## Find Count of Null, None, NaN of All DataFrame Columns

In [ ]:
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]
   ).show()

+-----------------+---------------------+------------+------------------+-----------------------+-------------------+------+-----------------------------+-------------------+-------------------+-----------------------+----------------+-----------+---------+
|Full Complaint ID|Complaint Year Number|Month Number|Record Create Date|Complaint Precinct Code|Patrol Borough Name|County|Law Code Category Description|Offense Description|PD Code Description|Bias Motive Description|Offense Category|Arrest Date|Arrest Id|
+-----------------+---------------------+------------+------------------+-----------------------+-------------------+------+-----------------------------+-------------------+-------------------+-----------------------+----------------+-----------+---------+
|                0|                    0|           0|                 0|                      0|                  0|     0|                            0|                  0|                  0|                      0|        

#### Get top 5 rows where complaint from Date is NaN

In [ ]:
df.where(col('Record Create Date').isNull()).show(n=5)

+-----------------+---------------------+------------+------------------+-----------------------+-------------------+------+-----------------------------+-------------------+-------------------+-----------------------+----------------+-----------+---------+
|Full Complaint ID|Complaint Year Number|Month Number|Record Create Date|Complaint Precinct Code|Patrol Borough Name|County|Law Code Category Description|Offense Description|PD Code Description|Bias Motive Description|Offense Category|Arrest Date|Arrest Id|
+-----------------+---------------------+------------+------------------+-----------------------+-------------------+------+-----------------------------+-------------------+-------------------+-----------------------+----------------+-----------+---------+
+-----------------+---------------------+------------+------------------+-----------------------+-------------------+------+-----------------------------+-------------------+-------------------+-----------------------+--------

In [ ]:
df.show(100)

+-----------------+---------------------+------------+------------------+-----------------------+--------------------+--------+-----------------------------+--------------------+--------------------+-----------------------+--------------------+-----------+---------+
|Full Complaint ID|Complaint Year Number|Month Number|Record Create Date|Complaint Precinct Code| Patrol Borough Name|  County|Law Code Category Description| Offense Description| PD Code Description|Bias Motive Description|    Offense Category|Arrest Date|Arrest Id|
+-----------------+---------------------+------------+------------------+-----------------------+--------------------+--------+-----------------------------+--------------------+--------------------+-----------------------+--------------------+-----------+---------+
|  201906112243017|                 2019|           3|        03/31/2019|                     61|PATROL BORO BKLYN...|   KINGS|                  MISDEMEANOR|CRIMINAL MISCHIEF...|CRIMINAL MISCHIEF...|

Check if minimum and maximum values of date and time are valid or not. The value of time cannot be 24:00:00. 

In [ ]:
## Minimum value of the column in pyspark
df.agg({'Record Create Date': 'min'}).show()

+-----------------------+
|min(Record Create Date)|
+-----------------------+
|             01/01/2019|
+-----------------------+



In [ ]:
## Maximum value of the column in pyspark
df.agg({'Record Create Date': 'max'}).show()

+-----------------------+
|max(Record Create Date)|
+-----------------------+
|             12/31/2019|
+-----------------------+



**Some basic data quality checks are as below:**
1. Check if there are no garbage values in the location of occurence description column. The valid values that this column should ideally contain are: 'FRONT OF', 'REAR OF', 'OUTSIDE', 'INSIDE', 'OPPOSITE OF'
2. Check if there are no garbage values in law category column. The valid values are: 'FELONY', 'VIOLATION', 'MISDEMEANOR'
3. Check if there are no misspellings in Borough Name. There should be 5 distinct boroughs: Manhattan, Bronx, Queens, Brooklyn, Staten Island. We implement unique method, in case of misspellings multiple values of the same borough would be returned.
4. Check if 'CRM_ATPT_CPTD_CD' column has no garbage value. The only acceptable values are Completed or Attempted.
5. Ideally, key code should contain only 3 digits. Implementing a check below to see if there are any invalid values for the key code. 

In [ ]:
df.columns

['Full Complaint ID',
 'Complaint Year Number',
 'Month Number',
 'Record Create Date',
 'Complaint Precinct Code',
 'Patrol Borough Name',
 'County',
 'Law Code Category Description',
 'Offense Description',
 'PD Code Description',
 'Bias Motive Description',
 'Offense Category',
 'Arrest Date',
 'Arrest Id']

In [ ]:
df.select('PD Code Description').distinct().show()

+--------------------+
| PD Code Description|
+--------------------+
|BURGLARY,UNCLASSI...|
|MISCHIEF, CRIMINA...|
|LARCENY,PETIT FRO...|
|MISCHIEF,CRIMINAL...|
|           ASSAULT 3|
|LARCENY,PETIT FRO...|
|LARCENY,GRAND FRO...|
|ASSAULT 2,1,UNCLA...|
|LARCENY,GRAND FRO...|
|ROBBERY,PUBLIC PL...|
|ROBBERY,COMMERCIA...|
|              RAPE 1|
|ROBBERY,PERSONAL ...|
|    ROBBERY,CLOTHING|
|MISCHIEF, CRIMINA...|
|ROBBERY,OPEN AREA...|
|MAKING TERRORISTI...|
|LARCENY,GRAND FRO...|
|ROBBERY,BEGIN AS ...|
| MURDER,UNCLASSIFIED|
+--------------------+
only showing top 20 rows



### Checks for Borough Name

In [ ]:
df.select('Patrol Borough Name').distinct().show()

+--------------------+
| Patrol Borough Name|
+--------------------+
|   PATROL BORO BRONX|
|PATROL BORO STATE...|
|PATROL BORO BKLYN...|
|PATROL BORO QUEEN...|
|PATROL BORO BKLYN...|
|PATROL BORO QUEEN...|
|PATROL BORO MAN N...|
|PATROL BORO MAN S...|
+--------------------+



We can see there are no misspellings for the Borough names and thus no need for additional data correction for the same.

In [ ]:
df.where(col('Patrol Borough Name').isNull()).show()

+-----------------+---------------------+------------+------------------+-----------------------+-------------------+------+-----------------------------+-------------------+-------------------+-----------------------+----------------+-----------+---------+
|Full Complaint ID|Complaint Year Number|Month Number|Record Create Date|Complaint Precinct Code|Patrol Borough Name|County|Law Code Category Description|Offense Description|PD Code Description|Bias Motive Description|Offense Category|Arrest Date|Arrest Id|
+-----------------+---------------------+------------+------------------+-----------------------+-------------------+------+-----------------------------+-------------------+-------------------+-----------------------+----------------+-----------+---------+
+-----------------+---------------------+------------+------------------+-----------------------+-------------------+------+-----------------------------+-------------------+-------------------+-----------------------+--------

#### Dropping Rows where Borough Name is NULL

In [ ]:
df.count()

1181

In [ ]:
df.filter(df.BORO_NM.isNull()).show()

+----------+------------+------------+------------+------------+-----------+------+-----+---------+-----+-------+----------------+----------+-------+-----------------+-------------+----------+-----------------+--------+----------+-----------+--------------+---------+--------+----------------+-----------+------------+-------------+--------+-------+
|CMPLNT_NUM|CMPLNT_FR_DT|CMPLNT_FR_TM|CMPLNT_TO_DT|CMPLNT_TO_TM|ADDR_PCT_CD|RPT_DT|KY_CD|OFNS_DESC|PD_CD|PD_DESC|CRM_ATPT_CPTD_CD|LAW_CAT_CD|BORO_NM|LOC_OF_OCCUR_DESC|PREM_TYP_DESC|JURIS_DESC|JURISDICTION_CODE|PARKS_NM|HADEVELOPT|HOUSING_PSA|SUSP_AGE_GROUP|SUSP_RACE|SUSP_SEX|TRANSIT_DISTRICT|PATROL_BORO|STATION_NAME|VIC_AGE_GROUP|VIC_RACE|VIC_SEX|
+----------+------------+------------+------------+------------+-----------+------+-----+---------+-----+-------+----------------+----------+-------+-----------------+-------------+----------+-----------------+--------+----------+-----------+--------------+---------+--------+----------------+-------

## Defining checks for outliers in age group

In [ ]:
df.show(n=5)

+-----------------+---------------------+------------+------------------+-----------------------+--------------------+--------+-----------------------------+--------------------+--------------------+-----------------------+--------------------+-----------+---------+
|Full Complaint ID|Complaint Year Number|Month Number|Record Create Date|Complaint Precinct Code| Patrol Borough Name|  County|Law Code Category Description| Offense Description| PD Code Description|Bias Motive Description|    Offense Category|Arrest Date|Arrest Id|
+-----------------+---------------------+------------+------------------+-----------------------+--------------------+--------+-----------------------------+--------------------+--------------------+-----------------------+--------------------+-----------+---------+
|  201906112243017|                 2019|           3|        03/31/2019|                     61|PATROL BORO BKLYN...|   KINGS|                  MISDEMEANOR|CRIMINAL MISCHIEF...|CRIMINAL MISCHIEF...|

#### Get unique values of offense description in sorted order

In [ ]:
df.select('Offense Description').distinct().orderBy('Offense Description', ascending=True).show()

+--------------------+
| Offense Description|
+--------------------+
|ASSAULT 3 & RELAT...|
|            BURGLARY|
|CRIMINAL MISCHIEF...|
|   CRIMINAL TRESPASS|
|   DANGEROUS WEAPONS|
|      FELONY ASSAULT|
|              FRAUDS|
|       GRAND LARCENY|
|       HARRASSMENT 2|
|INVESTIGATIONS/CO...|
|MISCELLANEOUS PEN...|
|MURDER & NON-NEGL...|
|OFF. AGNST PUB OR...|
|       PETIT LARCENY|
|                RAPE|
|             ROBBERY|
|          SEX CRIMES|
+--------------------+



#### Getting Total Count of Offense Description

In [ ]:
df.select('Offense Description').distinct().count()

17

In [ ]:
df.groupBy('Offense Description').count().show()

+--------------------+-----+
| Offense Description|count|
+--------------------+-----+
|OFF. AGNST PUB OR...|  152|
|      FELONY ASSAULT|  136|
|CRIMINAL MISCHIEF...|  310|
|MURDER & NON-NEGL...|    2|
|MISCELLANEOUS PEN...|  292|
|          SEX CRIMES|    3|
|       HARRASSMENT 2|   15|
|   CRIMINAL TRESPASS|    1|
|ASSAULT 3 & RELAT...|  204|
|              FRAUDS|    1|
|             ROBBERY|   37|
|   DANGEROUS WEAPONS|    1|
|       PETIT LARCENY|    2|
|                RAPE|    1|
|       GRAND LARCENY|   17|
|INVESTIGATIONS/CO...|    1|
|            BURGLARY|    6|
+--------------------+-----+



#### Each key code represents a particular offense description. There is a one to one mapping. So we would use key code for future analysis instead of offense description.

#### Calculating the null values present in the data columnwise (with respect to the features)

In [ ]:
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+-----------------+---------------------+------------+------------------+-----------------------+-------------------+------+-----------------------------+-------------------+-------------------+-----------------------+----------------+-----------+---------+
|Full Complaint ID|Complaint Year Number|Month Number|Record Create Date|Complaint Precinct Code|Patrol Borough Name|County|Law Code Category Description|Offense Description|PD Code Description|Bias Motive Description|Offense Category|Arrest Date|Arrest Id|
+-----------------+---------------------+------------+------------------+-----------------------+-------------------+------+-----------------------------+-------------------+-------------------+-----------------------+----------------+-----------+---------+
|                0|                    0|           0|                 0|                      0|                  0|     0|                            0|                  0|                  0|                      0|        

In [ ]:
amount_missing_df = df.select([(count(when(isnan(c) | col(c).isNull(), c))/count(lit(1))).alias(c) for c in df.columns])
amount_missing_df.show()

+-----------------+---------------------+------------+------------------+-----------------------+-------------------+------+-----------------------------+-------------------+-------------------+-----------------------+----------------+-----------------+-----------------+
|Full Complaint ID|Complaint Year Number|Month Number|Record Create Date|Complaint Precinct Code|Patrol Borough Name|County|Law Code Category Description|Offense Description|PD Code Description|Bias Motive Description|Offense Category|      Arrest Date|        Arrest Id|
+-----------------+---------------------+------------+------------------+-----------------------+-------------------+------+-----------------------------+-------------------+-------------------+-----------------------+----------------+-----------------+-----------------+
|              0.0|                  0.0|         0.0|               0.0|                    0.0|                0.0|   0.0|                          0.0|                0.0|          

COUNTY wise count

In [ ]:
df.groupBy('County').count().show()

+--------+-----+
|  County|count|
+--------+-----+
|   KINGS|  403|
|  QUEENS|  215|
|   BRONX|   95|
|RICHMOND|   47|
|NEW YORK|  421|
+--------+-----+



### Number of columns in Clean Data

In [ ]:
len(df.columns)

14

### Number of rows in Clean Data

In [ ]:
df.count()

1181

In [ ]:
df.printSchema()

root
 |-- Full Complaint ID: long (nullable = true)
 |-- Complaint Year Number: integer (nullable = true)
 |-- Month Number: integer (nullable = true)
 |-- Record Create Date: string (nullable = true)
 |-- Complaint Precinct Code: integer (nullable = true)
 |-- Patrol Borough Name: string (nullable = true)
 |-- County: string (nullable = true)
 |-- Law Code Category Description: string (nullable = true)
 |-- Offense Description: string (nullable = true)
 |-- PD Code Description: string (nullable = true)
 |-- Bias Motive Description: string (nullable = true)
 |-- Offense Category: string (nullable = true)
 |-- Arrest Date: string (nullable = true)
 |-- Arrest Id: string (nullable = true)



### **Exporting Clean Data in CSV**

The Cleaned Data Set will be saved as `NYPD_Complaint_Data_Historic_Cleaned.csv`

In [ ]:
pd_df = df.toPandas()
pd_df.to_csv("NYPD_Complaint_Data_Historic_Cleaned_Spark.csv")